# 문제 6

(kaggle 형 문제) prob4 를 바탕으로 아래 데이터셋을 만든다.

. prob6_train: DateHour 변수 기준으로 2021년 8월 14일 전(8월 14일 미포함) 데이터 (행의 수: 5256개)

. prob6_test: DateHour 변수 기준으로 2021년 8월 14일 이후(8월 14일 포함) 데이터 (행의 수: 744 개)

일 때, prob6_train으로 target을 예측하는 모델을 만들어, 

prob6_test의 target에 대한 MAE를 최소화하는 모델을 만든다. 

prob6_test의 예측 결과를 아래와 같은 형식으로 출력한다. 파일명은 answer6.csv 이다.

|DateHour|TotalHour|
|--------|---------|
|2021-08-14 00:00:00|102.607580|
|2021-08-14 01:00:00|94.078890|
....

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1


**문제1에서 5까지에서 필요한 내용들을 뽑아 옵니다.**

In [2]:
# parse_dates를 통해 datetime 형식으로 불러옵니다.
df_elec = pd.read_csv('elec.csv', parse_dates=['Date', 'DateHour'])
df_elec1 = df_elec.set_index(['DateHour', 'Minute']).drop(columns=['Date']).unstack()
min_cols = ['15분', '30분', '45분', '60분']
df_elec1.columns = min_cols
holi = pd.to_datetime(["2021-01-01", "2021-02-11", "2021-02-12", "2021-03-01", "2021-05-05", "2021-05-19", "2021-08-16"]).date
df_elec1 = df_elec1.assign(
    DayName = df_elec1.index.weekday, # 요일을 뽑아옵니다
    Hour = df_elec1.index.hour, # 시간을 뽑아옵니다.
    AM = (df_elec1.index.hour >= 12).astype(int),
    Weekend_yn = df_elec1.index.weekday.isin([5, 6]).astype(int), # isin을 통해 리스트 안의 값에 해당하는지 봅니다.
    Holiday_yn = np.isin(df_elec1.index.date, holi).astype(int), # 일자만을 뽑아 비교합니다.
    Avg = df_elec1[min_cols].mean(axis=1), # 평균 
    TotalHour = df_elec1[min_cols].sum(axis=1) # 합계
)
df_elec1.head()

,15분,30분,45분,60분,DayName,Hour,AM,Weekend_yn,Holiday_yn,Avg,TotalHour
DateHour,,,,,,,,,,,
2021-01-01 00:00:00,62,61,61,61,4,0,0,0,1,61.25,245
2021-01-01 01:00:00,96,93,116,113,4,1,0,0,1,104.50,418
2021-01-01 02:00:00,106,96,106,107,4,2,0,0,1,103.75,415
2021-01-01 03:00:00,92,110,110,109,4,3,0,0,1,105.25,421
2021-01-01 04:00:00,108,105,106,108,4,4,0,0,1,106.75,427


In [3]:
df_info = pd.read_csv('info.csv')
# pd.to_datetime으로 문자열을 datetime 형식으로 바꿔 봅니다.
df_info['Date'] = pd.to_datetime(df_info['Date'])
df_info1 = df_info.fillna(0) # 결측을 모두 0으로 대치하고 info1에 넣습니다.
df_info1.head()

,Date,생산량,기온,풍속,습도,강수량,전기요금(계절),공장인원,인건비
0,2021-01-01 00:00:00,0,-3.2,2.4,71,0.0,109.8,0.0,1.5
1,2021-01-01 01:00:00,0,-4.5,1.5,77,0.0,109.8,0.0,1.5
2,2021-01-01 02:00:00,0,-3.9,2.6,58,0.0,109.8,0.0,1.5
3,2021-01-01 03:00:00,0,-4.1,2.6,56,0.0,109.8,0.0,1.5
4,2021-01-01 04:00:00,0,-4.6,2.6,60,0.0,109.8,0.0,1.5


In [4]:
# 방법1: merge
# merge는 caller(호출자, left_on), callee(피호출자, right_on)을 컬럼(인덱스에 명칭이 부여됐을 경우, 인덱스 사용가)을
# 기준으로 결합니다.
df_basetable1 = df_elec1.merge(df_info1, left_on='DateHour', right_on='Date', how='inner')\
        .rename(columns={'Date': 'DateHour'}).set_index('DateHour')
df_basetable1.shape

(6168, 19)

In [5]:
df_prob3 = df_basetable1.sort_index()
df_prob3['target'] = df_basetable1['TotalHour'].shift(-24)
df_prob3 = df_prob3.dropna()
df_prob4 = pd.concat(
    [df_prob3] + 
    [df_prob3['TotalHour'].shift(24 * i).rename('lag_{}'.format(i)) for i in range(1, 7)],
axis=1).dropna()

In [6]:
df_prob4.isna().sum()

15분           0
30분           0
45분           0
60분           0
DayName       0
Hour          0
AM            0
Weekend_yn    0
Holiday_yn    0
Avg           0
TotalHour     0
생산량           0
기온            0
풍속            0
습도            0
강수량           0
전기요금(계절)      0
공장인원          0
인건비           0
target        0
lag_1         0
lag_2         0
lag_3         0
lag_4         0
lag_5         0
lag_6         0
dtype: int64

In [7]:
df_prob6_train = df_prob4.loc[df_prob4.index < '2021-08-14'].copy() # 실제 학습셋으로 간주합니다.
df_prob6_test = df_prob4.loc[df_prob4.index >= '2021-08-14'].copy() # 실제 평가셋으로 간주합니다.

**여기까지가 뽑아온 부분의 마지막입니다.**

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

In [9]:
# train/test split 방법을 사용하여 검증해보겠습니다.
df_train = df_prob6_train.loc[df_prob6_train.index < '2021-07-14'].copy()
df_test = df_prob6_train.loc[df_prob6_train.index >= '2021-07-14'].copy()

In [10]:
# 베이스라인 모델로 SVR 사용합니다.
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
# 문제 3에서의 모델을 가져와 봅니다.
X_svr = ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]
# 표준화 처리기를 앞단에 위치시킵니다.
reg_svr = make_pipeline(StandardScaler(), SVR(kernel='rbf', C=10, gamma=0.1))
reg_svr.fit(df_train[X_svr], df_train['target'])
# 학습 성능입니다.
mean_absolute_error(df_train['target'], reg_svr.predict(df_train[X_svr]))

61.50534596142382

In [11]:
# 테스트 성능입니다.
mean_absolute_error(df_test['target'], reg_svr.predict(df_test[X_svr]))

148.65000821712516

In [12]:
# 학습셋 전체로 학습을 시킵니다.
reg_svr.fit(df_prob6_train[X_svr], df_prob6_train['target'])
# 제출 파일을 만듭니다.
# to_csv의 index 매개변수로, 인덱스 포함여부를 정할 수 있습니다. index=None을하면 인덱스를 기록하지 않습니다.
# pd.DataFrame으로 데이터프레임을 만들 때 index를 포함하지 않더라도, index=None을 하지 않으면
# 디폴트 인덱스인 0, 1, 2, ... 가 출력됩니다.
# answer6.csv를 반드시 확인하세요.
pd.DataFrame(
    reg_svr.predict(df_prob6_test[X_svr]),
    columns=['target'],
    index=df_prob6_test.index
).to_csv('answer6.csv')

In [13]:
from sklearn.preprocessing import OneHotEncoder
X_cont = ['TotalHour'] + ['lag_{}'.format(i) for i in range(1, 7)]
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first', categories='auto'), ['DayName']),
    ('std', StandardScaler(), X_cont)
])
X_svr_2 = X_cont + ['DayName']
reg_svr_2 = make_pipeline(ct, SVR(kernel='rbf', C=10, gamma=0.1))
reg_svr_2.fit(df_train[X_svr_2], df_train['target'])
mean_absolute_error(df_train['target'], reg_svr_2.predict(df_train[X_svr_2]))

59.927341903643075

In [14]:
mean_absolute_error(df_test['target'], reg_svr_2.predict(df_test[X_svr_2]))

146.98340618482405

In [15]:
from sklearn.ensemble import GradientBoostingRegressor
X_gb ="15분, 30분, 45분, 60분, DayName, Hour, AM, Weekend_yn, Holiday_yn, Avg, TotalHour, 생산량, 기온, 풍속, 습도, 강수량, 전기요금(계절), 공장인원, 인건비"
X_gb =[i.strip() for i in X_gb.split(',')]
# Voting을 위해 앞단에 feature filter 셋팅으로 ColumnTransformer를 셋팅한 것입니다.
reg_gb = make_pipeline(
    ColumnTransformer([
        ('pt', 'passthrough', X_gb)
    ]),
    GradientBoostingRegressor(
        loss='huber', # 손실함수: huber
        n_estimators=75, # 트리의 수: 75
        max_depth=7, # 트리의 최대 깊이: 7
        min_samples_leaf=4, # 리프 노드(Leaf Node)의 최소 샘플 수: 4
        random_state=123 # 랜덤 시드: 123
    )
)
reg_gb.fit(df_train[X_gb], df_train['target'])
mean_absolute_error(df_train['target'], reg_gb.predict(df_train[X_gb]))

36.73714649971901

In [16]:
# SVR보다 성능이 개선 됐습니다.
mean_absolute_error(df_test['target'], reg_gb.predict(df_test[X_gb]))

117.32391481946111

In [17]:
from sklearn.ensemble import VotingRegressor
# reg_svr_2 모델과 reg_gb 모델을 결합한 VotingRegressor 앙상블 모델을 만듭니다.
reg_vt = VotingRegressor([
    ('reg_svr_2', reg_svr_2),
    ('reg_gb', reg_gb)
])
X_vt = X_gb + ['lag_{}'.format(i) for i in range(1, 7)]
reg_vt.fit(df_train[X_vt], df_train['target'])
mean_absolute_error(df_train['target'], reg_vt.predict(df_train[X_vt]))

45.54555120559154

In [18]:
# 성능이 개선 되진 않았습니다.
mean_absolute_error(df_test['target'], reg_vt.predict(df_test[X_vt]))

128.5001623335758

In [19]:
# reg_gb를 최종 Output으로 결정합니다.

reg_gb.fit(df_prob6_train[X_gb], df_prob6_train['target'])
pd.DataFrame(
    reg_gb.predict(df_prob6_test[X_gb]),
    columns=['target'],
    index=df_prob6_test.index
).to_csv('answer6.csv')

**실제 성능을 평가해 봅니다.**

In [20]:
mean_absolute_error(df_prob6_test['target'], reg_gb.predict(df_prob6_test[X_gb]))

38.86815607791259